<a href="https://colab.research.google.com/github/Gayaz2000/Redis_Test/blob/main/redis_caching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install langchain_community langchain_core langchain_redis redis langchain_groq langchain_ollama

In [4]:
import os
import getpass

os.environ["GROQ_API_KEY"] = getpass.getpass("Enter Groq api key: ")

Enter Groq api key: ··········


### Load dependecies

In [3]:
import time
import redis

from langchain_core.globals import set_llm_cache
from langchain_groq import ChatGroq
from langchain_redis import RedisCache, RedisSemanticCache
from langchain_ollama import OllamaEmbeddings

### Connecting to Redis

In [ ]:
REDIS_URL = getpass.getpass("Enter Redis URL: ") #"redis://localhost:6379"
redis_client = redis.from_url(REDIS_URL)
redis_client.ping()

### Using Redis for Standard Cache

In [ ]:
redis_cache = RedisCache(redis_url=REDIS_URL)
set_llm_cache(redis_cache)

llm = ChatGroq(model="gemma2-9b-it")

def execute_with_timing(prompt):
  start_time = time.time()
  result = llm.invoke(prompt)
  end_time = time.time()
  return result, end_time - start_time

#### Usecase

In [ ]:
prompt = ""
result, time_taken = execute_with_timing(prompt)
print(f"Result: {result}")
print(f"Time taken: {time_taken}")

In [ ]:
prompt = ""
result, time_taken = execute_with_timing(prompt)
print(f"Result: {result}")
print(f"Time taken: {time_taken}")

In [ ]:
# redis_cache.clear() # removes stored cache

### Using Redis Semantic Caching

In [ ]:
embedding = OllamaEmbeddings(model="llama3.2:1b")
redis_semantic_cache = RedisSemanticCache(
    embeddings= embedding,
    redis_url=REDIS_URL,
    distance_threshold= 0.2
    )

set_llm_cache(redis_semantic_cache)

#### Usecase

In [ ]:
prompt = ""
result, time_taken = execute_with_timing(prompt)
print(f"Result: {result}")
print(f"Time taken: {time_taken}")

In [ ]:
prompt = ""
result, time_taken = execute_with_timing(prompt)
print(f"Result: {result}")
print(f"Time taken: {time_taken}")

In [ ]:
# redis_semantic_cache.clear()

## Chat Message History using redis

In [ ]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_redis import RedisChatMessageHistory

session_id = ""

redis_message_history = RedisChatMessageHistory(
    session_id=session_id,
    redis_url=REDIS_URL,
)

In [ ]:
redis_message_history.add_user_message("Hi, AI system")
redis_message_history.add_ai_message("Hello!, How can I help you.") # Response for query

In [ ]:
redis_message_history.messages

### Chatbot integration

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

chain = prompt | llm

In [ ]:
def get_redis_chat_history(session_id: str)-> BaseChatMessageHistory:
  return RedisChatMessageHistory(session_id=session_id, redis_url= REDIS_URL)

In [ ]:
chain_with_chat_history = RunnableWithMessageHistory(
    chain,
    get_session_history= get_redis_chat_history,
    input_messages_key= "input",
    history_messages_key="history"
)

response = chain_with_chat_history.invoke(
    {"input": "Hi, AI system"},
    config={"configurable":{"session_id":"1"}}
)
print(response)

In [ ]:
redis_message_history.clear()